# simulation with ivs data

### Imports
Import the required libraries

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os
import pathlib

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from shapely.geometry import Point
import shapely
import geopandas as gpd
import movingpandas as mpd

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pickle

# OpenTNSIM
import opentnsim
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot


# dtv_backend
import dtv_backend.fis as fis
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.postprocessing
import dtv_backend.simple
import dtv_backend.simulate

# Used for mathematical functions
import math             
import numpy as np

# Used for making the graph to visualize our problem
import networkx as nx  
import requests
import io

### Create graph

In [2]:
#Load lock info
url = "https://zenodo.org/records/6673604/files/FIS_locks_grouped.geojson?download=1"
resp = requests.get(url)
stream = io.BytesIO(resp.content)
locks_gdf = gpd.read_file(stream)

In [3]:
#Load graph
url = "https://zenodo.org/record/6673604/files/network_digital_twin_v0.3.pickle?download=1"
graph1 = dtv_backend.fis.load_fis_network(url).copy()

# #remove edges without classification
# for edge in graph1.edges:
#     if str(graph1.edges[edge]['Classification']) == 'nan':
#         graph1.remove_edge(edge[0], edge[1])

graph = graph_module.Graph()
graph.graph = graph1
graph.graph_info = opentnsim.utils.info(graph.graph)

In [5]:
# save graph in geopanda
df_edges = nx.to_pandas_edgelist(graph1)
gpd.GeoDataFrame(df_edges).to_file('plots_routes/edges_zonder_afzetting')


C:\Users\hemert\AppData\Local\Temp\ipykernel_14068\3461875249.py:3: UserWarning:

Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.



### Make vessels and paths

In [6]:
import pickle
data = pickle.load(open(pathlib.Path("~/data/ivs/ivs-2024-geocoded_filtered.pkl").expanduser(), 'rb'))

##### read data

In [7]:
# # lees data in.
# data = gpd.read_file("data\ivs\ivs-2024-geocoded.gpkg")

# #filter data op bestaande iso datum en geometry.
# data['datetime'] = pd.to_datetime(data['v05_06_begindt_evenement_iso'], format = 'ISO8601', errors = 'coerce')
# data.dropna(subset = ['datetime', 'geometry'], inplace = True)


filter only on rotterdam to duisburg, 2023. Only keep ships in january 2023

In [8]:
# t_begin = pd.Timestamp('2023-12-25', tz='UTC')
# t_end = pd.Timestamp('2023-12-26', tz='UTC')
# #data1 = data[
#     #(data.UNLO_herkomst== "NLDOR")
#     #(data.UNLO_herkomst== "NLGAM")
#     #((data.UNLO_bestemming.str.startswith("DE")) | (data.UNLO_herkomst.str.startswith("DE")))
#     #  & (data.v05_06_begindt_evenement_iso.str.startswith("2023-01"))
#     #  ]
# condition_1 = (data.UNLO_bestemming.str.startswith("DE")) | (data.UNLO_herkomst.str.startswith("DE"))
# condition_2 = data.datetime>=(t_begin)
# condition_3 = data.datetime<(t_end)

# idx = np.logical_and.reduce([
#  condition_1,
#  condition_2,
#  condition_3
# ])
# data = data[idx]
# len(data)


##### Create vessels

In [9]:
import functools
from pandas.api.types import CategoricalDtype

def path_restricted_to_cemt_class(graph, origin, destination, ship_cemt_classe):
    """find a path restricted to allowed cemt classes

    Parameters
    ----------
    graph : networkx.Graph
        graph in which to find a path. graph edges should have information 'cemt'.
    origin : str
        origin node id
    destination : str
        destination node id
    ship_cemt_classe : str
        cemt class of the ship.
    """
    # create function to compute weights for this ship
    compute_weight = functools.partial(
        __compute_weight, ship_cemt_classe=ship_cemt_classe
    )

    # find the path
    path = nx.dijkstra_path(graph, origin, destination, weight=compute_weight)
    return path


def path_restricted_to_rws_class(graph, origin, destination, ship_rws_classe):
    """find a path restricted to allowed cemt classes

    Parameters
    ----------
    graph : networkx.Graph
        graph in which to find a path. graph edges should have information 'cemt'.
    origin : str
        origin node id
    destination : str
        destination node id
    ship_cemt_classe : str
        cemt class of the ship.
    """
    ship_cemt_classe = __scheepstype_rws_to_cemt(ship_rws_classe)
    return path_restricted_to_cemt_class(graph, origin, destination, ship_cemt_classe)


def __compute_weight(origin, target, dictionary_edge, ship_cemt_classe):
    # order classes from smallest to largest
    cemt_classes = CategoricalDtype(categories = ["0", "I", "II", "III", "IV", "IVa", "Va", "Vb", "VIa", "VIb", "VIc", "VIIa"]
                                ,ordered=True)
    #define synonyms:
    code_synonyms = {
        "_0": "0",
        "V_A": "Va",
        "V_B": "Vb",
        "VI_A": "VIa",
        "VI_B": "VIb",
        "VI_C": "VIc",
    }
    if dictionary_edge["Code"] in code_synonyms:
        edge_code= code_synonyms[dictionary_edge["Code"]]
    else:
        edge_code = dictionary_edge["Code"]
        
    # create codes
    codes = pd.Series(data=[edge_code, ship_cemt_classe], 
                        index=['edge', 'ship'], 
                        dtype=cemt_classes
                        )
    if codes.isna().any():
        return dictionary_edge["length_m"]
    if codes["edge"] < codes["ship"]:
        return np.nan
    else:
        return dictionary_edge["length_m"]


def __scheepstype_rws_to_cemt(rws_classe):
    rws_to_cemt = {
        "M0": "0",
        "M1": "I",
        "M2": "II",
        "M3": "III",
        "M4": "III",
        "M5": "III",
        "M6": "IVa",
        "M7": "IVa",
        "M8": "Va",
        "M9": "Va",
        "M10": "VIa",
        "M11": "VIa",
        "M12": "VIa",
        "M26": ?
    }
    return rws_to_cemt[rws_classe]


SyntaxError: invalid syntax (1818127684.py, line 92)

In [10]:
from networkx.exception import NetworkXNoPath
#from dtv_backend.fis import path_restricted_to_rws_class

# Make a class out of mix-ins
TransportResource = type('TransportResource', 
                         (core.Identifiable, core.ContainerDependentMovable, 
                          core.HasResource, core.Routable,
                          core.VesselProperties,
                         core.ExtraMetadata), 
                         {})
#define speed: 
def compute_v_provider(v_empty, v_full):
    return lambda x: 1

# define weight
def compute_weight(origin, target, dictionary):
    if dictionary['Code'] in ('IV', 'III', 'II', "I"): #niet langs te kleine wegen
        return None
    else:
        return dictionary['length_m']

vessels = []
failed_vessels = []
for index, row in data.iterrows():
    #determine path
    try:
        point_1 = fis.find_closest_node(graph.graph, Point(row.geometry.coords[0]))
        point_2 = fis.find_closest_node(graph.graph, Point(row.geometry.coords[-1]))
        # path = path_restricted_to_rws_class(graph = graph.graph, 
        #                                     origin = point_1[0], 
        #                                     destination=point_2[0], 
        #                                     ship_rws_classe=f"M{row['v15_1_Scheepstype_RWS']}") 
        path = nx.dijkstra_path(graph.graph, point_1[0], point_2[0], weight=compute_weight)
        #determine capacity
        capacity = max(row.v18_Laadvermogen*1000, row.v38_Vervoerd_gewicht, 1)
        data_vessel = {"env": None,
                "name": row.name,
                "type": row['v15_1_Scheepstype_RWS'],
                "B": 8,
                "L": 100,
                "route": path,
                "geometry": Point(row.geometry.coords[0]),  # lon, lat
                "capacity": capacity,
                "v": 0.5144*8, # 8 knopen
                "compute_v": compute_v_provider(v_empty=0.5144*8, v_full=0.5144*8),
                "departure_time": pd.to_datetime(row['v05_06_begindt_evenement_iso']),
                }
        vessel = TransportResource(**data_vessel)
        vessels.append(vessel)
    except NetworkXNoPath:
        failed_vessels.append(row.name)
    except ValueError:
        failed_vessels.append(row.name)
print(f"Failed vessels: {failed_vessels}")
        

# korte_route = nx.dijkstra_path(graph.graph, "8865735", "8861687")
# vessels = [TransportResource(**{
#     "env" : None,
#     "name": 'korte_route', 
#     "type": 'M6',
#     "B": 1, 
#     "L": 10,
#     "route": korte_route,
#     "geometry": Point(row.geometry.coords[0]),  # lon, lat
#     "capacity": capacity,
#     "v": 1,
#     "compute_v": compute_v_provider(v_empty=1, v_full=1),
# })]

Failed vessels: [464617, 659126, 797994, 835063, 1044412, 1561014, 1826422, 2138285, 2412099]


# sluis maken


### Start simulation

In [11]:
def start(env, vessel):
    while True:
        #wait untill ship will start sailing
        time_departure = time.mktime(vessel.metadata['departure_time'].timetuple())
        yield env.timeout(time_departure-env.now)

        # start sailing
        vessel.log_entry_v0("Start sailing", env.now, "", vessel.geometry)
        yield from vessel.move()
        vessel.log_entry_v0("Stop sailing", env.now, "", vessel.geometry)
        
        if vessel.geometry == nx.get_node_attributes(env.FG, "geometry")[vessel.route[-1]]:
            break

In [12]:
# Start simpy environment
simulation_start = min([vessel.metadata['departure_time'] for vessel in vessels])

env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
env.epoch = time.mktime(simulation_start.timetuple())


# Add graph to environment
#graph.add_resources(list(graph.graph.edges), np.ones(len(list(graph.graph.edges))), env)
env.FG = graph.graph



In [13]:
from dtv_backend.lock import Lock, Locks

locks = Locks(env)


In [14]:
import functools
for i, vessel in enumerate(vessels):
    # Add environment and path to the vessel
    vessel.env = env

    # add passing of a lock
    filled_pass_lock = functools.partial(locks.pass_lock_v2, vessel=vessel)
    vessel.on_pass_edge_functions = [filled_pass_lock]

    # Add the movements of the vessel to the simulation
    env.process(start(env, vessel))

env.run(until=env.timeout(60*60*24*30))

1703468791.0 s: 1779687 is waiting for lock L1306537_A
1703468791.0 s: 1779687 enters lock on side A
1703468851.0 s: lock 1306537 closes entry A
1703470651.0 s: lock 1306537 opens entry B
1703479591.0 s: 1306537 closes entry B
1703479591.0 s: 1046675 is waiting for lock L1306537_A
1703481391.0 s: 1306537 opens entry A
1703481391.0 s: 1046675 enters lock on side A
1703481511.0 s: lock 1306537 closes entry A
1703482751.7 s: 2244521 is waiting for lock L7069282_B
1703482811.7 s: lock 7069282 closes entry A
1703483191.0 s: 1950006 is waiting for lock L1306537_A
1703483311.0 s: lock 1306537 opens entry B
1703483431.0 s: 1306537 closes entry B
1703484611.7 s: lock 7069282 opens entry B
1703484611.7 s: 2244521 enters lock on side B
1703484731.7 s: 7069282 closes entry B
1703485231.0 s: 1306537 opens entry A
1703485231.0 s: 1950006 enters lock on side A
1703485351.0 s: lock 1306537 closes entry A
1703486531.7 s: 7069282 opens entry A
1703487151.0 s: lock 1306537 opens entry B
1703490391.0 s: 1

### Obtain vessel log information
The cel below uses the vessel log. The core function *log_entry* is used, which takes four arguments:

- **Log.** A text to describe what is logged.
- **t.** The timestamp.
- **Value.**  The value for the log (for sailing this is the distance).
- **Geometry** The location of the vessel while loggin.

In [15]:
log = gpd.GeoDataFrame()
for vessel in vessels:
    vessel_log = gpd.GeoDataFrame(vessel.logbook, geometry='Geometry')
    vessel_log['trajectory_id'] = f'vessel_{vessel.name}_trip_1'
    vessel_log['object_id'] = f'vessel_{vessel.name}'
    log = pd.concat([log, vessel_log])
mpd_log = mpd.TrajectoryCollection(log, traj_id_col='trajectory_id', obj_id_col='object_id', t='Timestamp')
mpd_log.to_line_gdf().to_file('plots_routes/duitsland_trajectories_zonder_afzetting_met_weight_function.gpkg')

c:\Users\hemert\Anaconda3\envs\trans2\Lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.

c:\Users\hemert\Anaconda3\envs\trans2\Lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.

c:\Users\hemert\Anaconda3\envs\trans2\Lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.

c:\Users\hemert\Anaconda3\envs\trans2\Lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.

c:\Users\hemert\Anaconda3\envs\trans2\Lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.

c:\Users\hemert\Anaconda3\envs\trans2\Lib\site-packages\movingpandas\t

In [16]:
lock_dfs = []
for id, lock_object in locks.locks_resources.items():
    lock = pd.DataFrame(lock_object.logbook)
    lock_properties = pd.DataFrame(lock["Value"].values.tolist())
    lock_df = pd.concat([lock, lock_properties], axis=1)
    lock_df['lock_id'] = id
    lock_dfs.append(lock_df)

all_locks_df = pd.concat(lock_dfs, axis=0)
all_locks_df = all_locks_df.drop(columns=['Value'])
all_locks_df.to_csv('plots_routes/trajectories_zonder_afzetting_met_weight_function_locks.csv')

### Visualization of path
If you get an error regarding ffmpeg use [this](https://stackoverflow.com/questions/13316397/matplotlib-animation-no-moviewriters-available) answer. You have to install ffmpeg in your Conda environment. It can be done using the following command.

```bash
#Install ffmpeg using Conda
conda install -c conda-forge ffmpeg
```

In [ ]:
# visualise vessel movements based on the information included in the vessel.log
plot.vessel_kml(env, vessels, stepsize = 60)
#plot.graph_kml(env)


In [ ]:
data.to_pickle('data/ivs/ivs-2024-geocoded_filtered.pkl')

In [ ]:
data = pickle.load(open('data/ivs/ivs-2024-geocoded_filtered.pkl', 'rb'))

In [ ]:
! start explorer .